<a href="https://colab.research.google.com/github/jinsusong/study-Kaggle/blob/main/QA_TensorFlow_2_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 2.0 Question Answering

#데이터 로드

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c tensorflow2-question-answering

In [ ]:
%cd /content

In [ ]:
%pwd

In [ ]:
!mkdir data


In [ ]:
!unzip simplified-nq-train.jsonl.zip -d data/

In [ ]:
!unzip simplified-nq-test.jsonl.zip -d data/

In [ ]:
!pip install pysimdjson

- file.read_lines()은 한줄씩 읽어오기는 좋지만 input.txt의 내용이 기가바이트(gb)일 때는 일단 메모리에 다 올려놓고 처리 하기 때문에 데이터가 많다면 좋은 방법은 아닙니다.

- 메모리로 모두 읽은 후 처리하지 않고 한줄씩 읽어서 처리 합니다.

# 데이터 전처리 문장에서 <태그> 제거

In [ ]:
# 문단에서 html 제거를 위한 함수
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext =re.sub(cleanr,'',raw_html)
    return cleantext

# 데이터 전처리 json -> dataFrame

In [ ]:
def create_df(data):
        # X
        document = cleanhtml(data['document_text'])
        question = data['question_text']
        # Y
        y_start = data['annotations'][0]['long_answer']['start_token']
        y_end = data['annotations'][0]['long_answer']['end_token']

        if y_start == -1 :
            y_answer = "None"
        else:
            y_answer = cleanhtml(data['document_text'][y_start:y_end+1])

        #print(document)
        # print(question)
        # print(y_start)
        # print(y_end)
        # print(y_answer)

        temp_list = []
        temp_list.append(document)
        temp_list.append(question)
        temp_list.append(y_answer)
        return temp_list




In [ ]:
import json
from pprint import pprint
import re
import pandas as pd


In [ ]:


file_path = "data/simplified-nq-train.jsonl"
i = 0 
comp_list = []
cols = ["text","question","answer"]

with open(file_path) as f:
    for line in f:
        data = json.loads(line)
        new_df = create_df(data)
        comp_list.append(new_df)

new_df = pd.DataFrame(comp_list, columns=cols)

- X data
1. document_text : str : 기사   
2. question_text : str : 질문 
3. long_answer_candidates : list : 긴 답변 후보들 : ??? 
    - start_token : int , 
    - end_token : int , 
    - top_level: True/False 

- Y label
1. annotations : list : 올바른 긴 답변 , 짧은 답변
    - annotation_id, 
    - long_answer, 
        - candidate_index
        - start_token : int
        - end_token : int
    - short_answers, 
        - start_token : int 
        - end_token : int
    - yes_no_answer

In [ ]:
# pprint(data.keys())
# print(pprint(data))
# print(data['annotations'][0]['long_answer']['start_token'])
# print(data['annotations'][0]['yes_no_answer'])
# print(len(data))

In [ ]:
new_df.head()

In [ ]:
new_df.to_csv("tf_QA_data.csv", index=False)

In [ ]:
data = pd.read_csv("tf_QA_data.csv")
data.head()

In [ ]:
len(data)

#모델링 

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, TFBertForQuestionAnswering

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForQuestionAnswering.from_pretrained('bert-base-cased')

In [ ]:

import tensorflow as tf

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, return_tensors='tf')
outputs = model(input_dict)
start_logits = outputs.start_logits
end_logits = outputs.end_logits

all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

In [ ]:
answer